In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import BinaryF1Score

from collections import Counter
import matplotlib.pyplot as plt

c:\Users\PNC\anaconda3\envs\Project_38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_PATH = "./reshaped_data.csv"
LABEL_PATH = "./Label.csv"

data_df = pd.read_csv(DATA_PATH)
label_df = pd.read_csv(LABEL_PATH)

In [3]:
print(data_df.head())
print()
print()
print(label_df.head())

   frame  source_index       x_0      x_11      x_12      x_13      x_14  \
0    1.0             0  0.432988  0.465422  0.408942  0.474997  0.391634   
1    2.0             0  0.432988  0.465422  0.408942  0.474997  0.391634   
2    3.0             0  0.432988  0.465422  0.408942  0.474997  0.391634   
3    4.0             0  0.432988  0.465422  0.408942  0.474997  0.391634   
4    5.0             0  0.432988  0.465422  0.408942  0.474997  0.391634   

       x_15      x_16      x_23  ...      z_13      z_14      z_15      z_16  \
0  0.438641  0.427603  0.450574  ... -0.326235 -0.350726 -0.429115 -0.415761   
1  0.438641  0.427603  0.450574  ... -0.326235 -0.350726 -0.429115 -0.415761   
2  0.438641  0.427603  0.450574  ... -0.326235 -0.350726 -0.429115 -0.415761   
3  0.438641  0.427603  0.450574  ... -0.326235 -0.350726 -0.429115 -0.415761   
4  0.438641  0.427603  0.450574  ... -0.326235 -0.350726 -0.429115 -0.415761   

       z_23      z_24      z_25      z_26      z_29      z_30 

In [4]:
data_df["source_index"].value_counts()


source_index
2652    692
2655    668
2653    640
2651    545
2648    534
       ... 
1109    300
1110    300
1111    300
1112    300
3270    300
Name: count, Length: 2950, dtype: int64

In [5]:
val = data_df["source_index"].value_counts().values
idx = data_df["source_index"].value_counts().index

for i, num in enumerate(val):
    if num > 300:
        data_df[data_df["source_index"] == idx[i]] = data_df[data_df["source_index"] == idx[i]][:300]

data_df["source_index"].value_counts()

source_index
0.0       300
2179.0    300
2170.0    300
2171.0    300
2172.0    300
         ... 
1114.0    300
1115.0    300
1116.0    300
1117.0    300
3270.0    300
Name: count, Length: 2950, dtype: int64

In [6]:
data_df.dropna(inplace = True)

In [7]:
X_list = []

for i in data_df["source_index"].unique():
    X_list.append(np.array(data_df[data_df["source_index"] == i].drop("source_index", axis = 1)))

X_list

[array([[1.00000000e+00, 4.32988077e-01, 4.65422332e-01, ...,
         9.63138640e-02, 2.85862595e-01, 2.88854957e-01],
        [2.00000000e+00, 4.32988077e-01, 4.65422332e-01, ...,
         9.63138640e-02, 2.85862595e-01, 2.88854957e-01],
        [3.00000000e+00, 4.32988077e-01, 4.65422332e-01, ...,
         9.63138640e-02, 2.85862595e-01, 2.88854957e-01],
        ...,
        [2.98000000e+02, 5.34380615e-01, 5.23442984e-01, ...,
         5.04104123e-02, 5.42477310e-01, 2.53765464e-01],
        [2.99000000e+02, 5.32720804e-01, 5.24880528e-01, ...,
         5.58799431e-02, 5.44007123e-01, 2.90664405e-01],
        [3.00000000e+02, 5.32224953e-01, 5.24520576e-01, ...,
         6.13131933e-02, 5.38691282e-01, 2.83639461e-01]]),
 array([[1.00000000e+00, 7.74211824e-01, 7.90820897e-01, ...,
         1.61929712e-01, 9.82979313e-02, 2.41869763e-01],
        [2.00000000e+00, 7.74211824e-01, 7.90820897e-01, ...,
         1.61929712e-01, 9.82979313e-02, 2.41869763e-01],
        [3.00000000e+00, 

In [8]:
y_list = list(label_df["label"])

y_list[-5:]

[0, 0, 0, 0, 0]

In [9]:
print(len(X_list), len(y_list))

2950 2950


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, 
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                    stratify = y_list)

In [11]:
len(X_train)

2360

In [12]:
class DetectDataset(Dataset):
    def __init__(self, feature, target):
        self.feature = feature
        self.target = target
        self.n_rows = len(self.feature)

    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        featureTS = torch.tensor(self.feature[index], dtype = torch.float32)
        targetTS = torch.tensor([self.target[index]], dtype = torch.float32)

        return featureTS, targetTS

In [13]:
BATCH_SIZE = 64

trainDS = DetectDataset(X_train, y_train)
trainDL = DataLoader(trainDS, batch_size = BATCH_SIZE, shuffle = True)

In [8]:
class FallDetectModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, 
                 dropout, bidirectional):
        super().__init__()

        self.model = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_dim,
            num_layers = n_layers,
            dropout = dropout,
            bidirectional = bidirectional,
            batch_first = True
        )

        if bidirectional:
            self.output = nn.Linear(hidden_dim * 2 , 1)
        
        else:
            self.output = nn.Linear(hidden_dim, 1)
    
    def forward(self, inputs):
        output, _ = self.model(inputs)
        output = output[:, -1, :]
        result = self.output(output)
        
        return result

In [9]:
EPOCH = 1000
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

input_size = 40
hidden_dim = 128
n_layers = 2
dropout = 0
bidirectional = True

fall_detect_model = FallDetectModel(input_size = input_size, 
                                    hidden_dim = hidden_dim,
                                    n_layers = n_layers,
                                    dropout = dropout,
                                    bidirectional = bidirectional).to(DEVICE)

In [16]:
fall_detect_model

FallDetectModel(
  (model): LSTM(40, 128, num_layers=2, batch_first=True, bidirectional=True)
  (output): Linear(in_features=256, out_features=1, bias=True)
)

In [17]:
F1score = BinaryF1Score().to(DEVICE)
BCELoss = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(fall_detect_model.parameters(), lr = LR)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = "max", patience = 10, verbose = True)

In [18]:
def testing(model, feature, target):
    featureTS = torch.FloatTensor(feature).to(DEVICE)
    targetTS = torch.FloatTensor(target).to(DEVICE)
    
    targetTS = targetTS.unsqueeze(1)

    model.eval()

    with torch.no_grad():
        pre_val = model(featureTS)
        loss_val = BCELoss(pre_val, targetTS)

        probs = torch.sigmoid(pre_val)
        preds = (probs > 0.5).int()

        score_val = F1score(preds, targetTS.int())
    
    print(len(preds))

    return loss_val, score_val, preds

In [19]:
def training(model, trainDL, X_test, y_test):
    SAVE_PATH = './saved_models/'
    os.makedirs(SAVE_PATH, exist_ok = True)

    BREAK_CNT_SCORE = 0
    LIMIT_VALUE = 50

    BCE_LOSS_HISTORY, SCORE_HISTORY = [[], []], [[], []]

    for epoch in range(1, EPOCH + 1):
        model.train()
        SAVE_WEIGHT = os.path.join(SAVE_PATH, f"model_weights_{epoch}.pth")

        bce_loss_total, score_total = 0, 0

        for featureTS, targetTS in trainDL:
            featureTS = featureTS.to(DEVICE)
            targetTS = targetTS.to(DEVICE)
            

            pre_val = model(featureTS)

            bce_loss = BCELoss(pre_val, targetTS)
            
            probs = torch.sigmoid(pre_val)
            preds = (probs > 0.5).int()

            score = F1score(preds, targetTS)
            
            bce_loss_total += bce_loss.item()
            score_total += score.item()

            optimizer.zero_grad()
            bce_loss.backward()
            optimizer.step()

        test_bce_loss, test_score, test_preds = testing(model, X_test, y_test)
        
        BCE_LOSS_HISTORY[0].append(bce_loss_total / len(trainDL))
        SCORE_HISTORY[0].append(score_total / len(trainDL))

        BCE_LOSS_HISTORY[1].append(test_bce_loss)
        SCORE_HISTORY[1].append(test_score)

        print(f"[{epoch} / {EPOCH}]\n - TRAIN BCE LOSS : {BCE_LOSS_HISTORY[0][-1]}")
        print(f"- TRAIN F1 SCORE : {SCORE_HISTORY[0][-1]}")

        print(f"\n - TEST BCE LOSS : {BCE_LOSS_HISTORY[1][-1]}")
        print(f"- TEST F1 SCORE : {SCORE_HISTORY[1][-1]}")

        scheduler.step(test_score)

        if len(SCORE_HISTORY[1]) >= 2:
            if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-2]: BREAK_CNT_SCORE += 1

        if len(SCORE_HISTORY[1]) == 1:
            torch.save(model.state_dict(), SAVE_WEIGHT)
        
        else:
            if SCORE_HISTORY[1][-1] > max(SCORE_HISTORY[1][:-1]):
                torch.save(model.state_dict(), SAVE_WEIGHT)

        if BREAK_CNT_SCORE > LIMIT_VALUE:
            print(f"성능 및 손실 개선이 없어서 {epoch} EPOCH에 학습 중단")
            break

    return BCE_LOSS_HISTORY, SCORE_HISTORY

In [20]:
bce_loss, f1_score = training(fall_detect_model, trainDL, X_test, y_test)

C:\Users\PNC\AppData\Local\Temp\ipykernel_14344\2077541451.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  featureTS = torch.FloatTensor(feature).to(DEVICE)


590
[1 / 1000]
 - TRAIN BCE LOSS : 0.6936832666397095
- TRAIN F1 SCORE : 0.5261189341545105

 - TEST BCE LOSS : 0.6903390884399414
- TEST F1 SCORE : 0.6917960047721863
590
[2 / 1000]
 - TRAIN BCE LOSS : 0.6885981253675513
- TRAIN F1 SCORE : 0.5840284196106164

 - TEST BCE LOSS : 0.6829150319099426
- TEST F1 SCORE : 0.6917960047721863
590
[3 / 1000]
 - TRAIN BCE LOSS : 0.6747024155951835
- TRAIN F1 SCORE : 0.5667933749186026

 - TEST BCE LOSS : 0.6250826120376587
- TEST F1 SCORE : 0.7455196976661682
590
[4 / 1000]
 - TRAIN BCE LOSS : 0.6443191315676715
- TRAIN F1 SCORE : 0.5394355419117052

 - TEST BCE LOSS : 0.6470154523849487
- TEST F1 SCORE : 0.695652186870575
590
[5 / 1000]
 - TRAIN BCE LOSS : 0.6074320722270656
- TRAIN F1 SCORE : 0.7794304473980053

 - TEST BCE LOSS : 0.3603639304637909
- TEST F1 SCORE : 0.8976377844810486
590
[6 / 1000]
 - TRAIN BCE LOSS : 0.4132986241901243
- TRAIN F1 SCORE : 0.8214910723067619

 - TEST BCE LOSS : 0.2744714617729187
- TEST F1 SCORE : 0.9022082090

In [21]:
for x, y in trainDL:
    print(x.shape)
    break

torch.Size([64, 300, 40])


In [62]:
video = r"./Test_Dataset/test_video.mp4"

import cv2
import mediapipe as mp




In [63]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=0,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    smooth_landmarks=True
)

important_landmarks = [
    0, 11, 12, 13, 14, 15, 16,
    23, 24, 25, 26, 29, 30
]

test_video = pd.DataFrame(columns = ["frame", "landmark_id", "x", "y", "z"])

frame_count = 0
processed_frame = 0

cap = cv2.VideoCapture(video)
original_fps = cap.get(cv2.CAP_PROP_FPS)
skip_interval = max(1, int(original_fps / 20))  # 20fps로 맞춤

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % skip_interval == 0:
        # frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        processed_frame += 1
        frame = cv2.resize(frame, (1640, 1280))
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            for idx in important_landmarks:
                landmark = results.pose_landmarks.landmark[idx]
                test_video.loc[len(test_video)] = [processed_frame, idx, landmark.x, landmark.y, landmark.z]

    frame_count += 1

cap.release()


In [64]:
test_video

,frame,landmark_id,x,y,z
0,83.0,0.0,0.497834,0.256350,-0.381163
1,83.0,11.0,0.529510,0.285437,-0.347989
2,83.0,12.0,0.509912,0.260619,-0.164051
3,83.0,13.0,0.523981,0.353358,-0.359943
4,83.0,14.0,0.510242,0.311787,-0.018255
...,...,...,...,...,...
2816,300.0,24.0,0.405087,0.540304,0.053838
2817,300.0,25.0,0.410869,0.637473,-0.057452
2818,300.0,26.0,0.375827,0.597249,0.067021
2819,300.0,29.0,0.453144,0.580940,0.124740


In [65]:
# 전체 프레임과 랜드마크 ID 목록 생성
all_frames = np.arange(1, 301)
all_landmarks = test_video['landmark_id'].unique()

df_frames = test_video["frame"].unique()

frame_list = []
landmark_list = []

for frame in all_frames:
    if frame not in df_frames:
        for landmark in all_landmarks:
            frame_list.append(frame)
            landmark_list.append(landmark)

add_row = pd.DataFrame({"frame" : frame_list, "landmark_id" : landmark_list})

test_video = pd.concat([test_video, add_row], ignore_index = True)

test_video = test_video.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop=True)
test_video = test_video.sort_values(by = ["frame", "landmark_id"])

if len(test_video) > 3900:
    test_video = test_video[:3900]

test_video

,frame,landmark_id,x,y,z
0,1.0,0.0,0.497834,0.256350,-0.381163
300,1.0,11.0,0.529510,0.285437,-0.347989
600,1.0,12.0,0.509912,0.260619,-0.164051
900,1.0,13.0,0.523981,0.353358,-0.359943
1200,1.0,14.0,0.510242,0.311787,-0.018255
...,...,...,...,...,...
2699,300.0,24.0,0.405087,0.540304,0.053838
2999,300.0,25.0,0.410869,0.637473,-0.057452
3299,300.0,26.0,0.375827,0.597249,0.067021
3599,300.0,29.0,0.453144,0.580940,0.124740


In [66]:
# frame 오름차순, source_index 우선 정렬
test_video = test_video.sort_values(by=["frame"])

# 피벗: (frame, source_index) 기준, landmark_id 별 x/y/z를 칼럼으로
df_pivot = test_video.pivot(index=["frame"], columns="landmark_id", values=["x", "y", "z"])

# 다중 인덱스 컬럼을 단일 열로 변환: ex) ('x', 11.0) -> x_11
df_pivot.columns = [f"{coord}_{int(lid)}" for coord, lid in df_pivot.columns]

# 인덱스 복구
df_pivot.reset_index(inplace=True)
                     
# 다시 source_index 기준으로 묶고, 그 안에서 frame 오름차순 정렬 (보장용)
df_pivot = df_pivot.sort_values(by=["frame"]).reset_index(drop=True)

df_pivot
 

,frame,x_0,x_11,x_12,x_13,x_14,x_15,x_16,x_23,x_24,...,z_13,z_14,z_15,z_16,z_23,z_24,z_25,z_26,z_29,z_30
0,1.0,0.497834,0.529510,0.509912,0.523981,0.510242,0.506634,0.484092,0.520244,0.508940,...,-0.359943,-0.018255,-0.385794,0.043837,-0.060830,0.060945,0.027731,0.162507,0.158319,0.314054
1,2.0,0.497834,0.529510,0.509912,0.523981,0.510242,0.506634,0.484092,0.520244,0.508940,...,-0.359943,-0.018255,-0.385794,0.043837,-0.060830,0.060945,0.027731,0.162507,0.158319,0.314054
2,3.0,0.497834,0.529510,0.509912,0.523981,0.510242,0.506634,0.484092,0.520244,0.508940,...,-0.359943,-0.018255,-0.385794,0.043837,-0.060830,0.060945,0.027731,0.162507,0.158319,0.314054
3,4.0,0.497834,0.529510,0.509912,0.523981,0.510242,0.506634,0.484092,0.520244,0.508940,...,-0.359943,-0.018255,-0.385794,0.043837,-0.060830,0.060945,0.027731,0.162507,0.158319,0.314054
4,5.0,0.497834,0.529510,0.509912,0.523981,0.510242,0.506634,0.484092,0.520244,0.508940,...,-0.359943,-0.018255,-0.385794,0.043837,-0.060830,0.060945,0.027731,0.162507,0.158319,0.314054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296.0,0.345862,0.397704,0.364439,0.423691,0.380274,0.406115,0.381651,0.431096,0.407100,...,-0.291196,-0.077108,-0.260406,-0.093144,-0.055727,0.055860,-0.073277,0.072378,0.112955,0.210925
296,297.0,0.345855,0.397663,0.364412,0.422906,0.379459,0.405165,0.382428,0.430844,0.407087,...,-0.281859,-0.075040,-0.244242,-0.092564,-0.055598,0.055740,-0.067080,0.079707,0.109183,0.209691
297,298.0,0.345792,0.396801,0.364299,0.420210,0.379412,0.404698,0.381608,0.429886,0.405873,...,-0.282308,-0.070819,-0.246455,-0.078057,-0.055552,0.055699,-0.064965,0.079696,0.109269,0.211621
298,299.0,0.346208,0.398207,0.362861,0.422705,0.379185,0.405798,0.381879,0.429869,0.405215,...,-0.283918,-0.075674,-0.250111,-0.084794,-0.053986,0.054111,-0.063691,0.061811,0.126405,0.210587


In [67]:
model = fall_detect_model

# 2. 그런 다음 state_dict를 로드합니다
model.load_state_dict(torch.load('./saved_models/model_weights_84.pth'))

C:\Users\PNC\AppData\Local\Temp\ipykernel_3156\921123962.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./saved_models/model_weights_8

<All keys matched successfully>

In [68]:
X_tensor = torch.tensor(df_pivot.values, dtype=torch.float32).to(DEVICE)
X_tensor = X_tensor.unsqueeze(0)

In [69]:
fall_detect_model.eval()
with torch.no_grad():
    output = fall_detect_model(X_tensor)  # 출력: (batch_size, 1)
    probs = torch.sigmoid(output)         # 확률화
    predicted_classes = (probs > 0.5).int()  # 0 또는 1로 변환

print("예측 결과:", predicted_classes)


예측 결과: tensor([[1]], device='cuda:0', dtype=torch.int32)
